**Importing** **Libraries**


In [ ]:
import os
import PIL
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0

**Importing the dataset from the drive** 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True) 

Mounted at /content/gdrive


In [ ]:
p="/content/gdrive/MyDrive/Colab Notebooks/Agriculture Vision Dataset/New Augmented"
os.chdir(p)
class_count = len(os.listdir(p))
print(class_count)

4


In [ ]:
batch_size=32
image_size=(512,512)

**Splitting** **The** **Dataset** **Into** **Training** **And** **Validation** **Data** 


In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/gdrive/MyDrive/Colab Notebooks/Agriculture Vision Dataset/New Augmented",
    validation_split=0.1,
    subset="training",
    seed=123,
    image_size=image_size,
    batch_size=32
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
   "/content/gdrive/MyDrive/Colab Notebooks/Agriculture Vision Dataset/New Augmented" ,
   validation_split=0.1,
   subset="validation",
   seed=123,
   image_size=image_size,
   batch_size=32
) 

Found 8000 files belonging to 4 classes.
Using 7200 files for training.
Found 8000 files belonging to 4 classes.
Using 800 files for validation.


**Importing The EfficientNet-B0 Model**

In [ ]:
base_model = EfficientNetB0(input_shape = (512, 512, 3), include_top = False, weights = None)


In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
flat = layers.Flatten()(base_model.output)
x = layers.Dense(512, activation='relu')(flat)
output = layers.Dense(4, activation='relu')(x)

model = tf.keras.models.Model(base_model.input, output)

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 512, 512, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 512, 512, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 513, 513, 3)  0           normalization[0][0]              
______________________________________________________________________________________________

**Experiment 1 - epochs=10 ,steps_per_epoch=10**

In [ ]:
reshist = model.fit(train_ds, validation_data = val_ds, steps_per_epoch = 10, epochs = 10)

Epoch 1/10
10/10 [==============================] - 586s 63s/step - loss: 1.3863 - accuracy: 0.2696 - val_loss: 1.3867 - val_accuracy: 0.2438
Epoch 2/10
10/10 [==============================] - 468s 50s/step - loss: 1.3867 - accuracy: 0.2451 - val_loss: 1.3867 - val_accuracy: 0.2438
Epoch 3/10
10/10 [==============================] - 471s 50s/step - loss: 1.3867 - accuracy: 0.2427 - val_loss: 1.3868 - val_accuracy: 0.2438
Epoch 4/10
10/10 [==============================] - 470s 50s/step - loss: 1.3884 - accuracy: 0.2146 - val_loss: 1.3864 - val_accuracy: 0.2438
Epoch 5/10
10/10 [==============================] - 482s 51s/step - loss: 1.3865 - accuracy: 0.2434 - val_loss: 1.3860 - val_accuracy: 0.2438
Epoch 6/10
10/10 [==============================] - 487s 52s/step - loss: 1.3852 - accuracy: 0.3078 - val_loss: 1.3860 - val_accuracy: 0.2438
Epoch 7/10
10/10 [==============================] - 473s 50s/step - loss: 1.3858 - accuracy: 0.2456 - val_loss: 1.3858 - val_accuracy: 0.2438
Epoch 

**Freezing All The Layers Of Model Except The Last 12 Layers**

In [ ]:
for layer in base_model.layers[0:-12]:
    layer.trainable = False

In [ ]:
flat = layers.Flatten()(base_model.output)
x = layers.Dense(512, activation='relu')(flat)
output = layers.Dense(4, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, output)

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

**Experiment 2 - epochs = 10,steps_per_epoch=10 (Freezing All The Layers Except Last 12 Layers Of The Baseline Model)**


In [ ]:
reshist = model.fit(train_ds, validation_data = val_ds, steps_per_epoch = 10, epochs = 10)

Epoch 1/10
10/10 [==============================] - 404s 42s/step - loss: 1.3863 - accuracy: 0.2779 - val_loss: 1.3874 - val_accuracy: 0.2488
Epoch 2/10
10/10 [==============================] - 378s 41s/step - loss: 1.3867 - accuracy: 0.2481 - val_loss: 1.3868 - val_accuracy: 0.2488
Epoch 3/10
10/10 [==============================] - 386s 41s/step - loss: 1.3853 - accuracy: 0.2281 - val_loss: 1.3864 - val_accuracy: 0.2362
Epoch 4/10
10/10 [==============================] - 390s 42s/step - loss: 1.3865 - accuracy: 0.2338 - val_loss: 1.3865 - val_accuracy: 0.2362
Epoch 5/10
10/10 [==============================] - 384s 41s/step - loss: 1.3866 - accuracy: 0.2353 - val_loss: 1.3861 - val_accuracy: 0.2713
Epoch 6/10
10/10 [==============================] - 399s 43s/step - loss: 1.3857 - accuracy: 0.2731 - val_loss: 1.3854 - val_accuracy: 0.2438
Epoch 7/10
10/10 [==============================] - 395s 43s/step - loss: 1.3854 - accuracy: 0.2529 - val_loss: 1.3848 - val_accuracy: 0.2537
Epoch 

**Experiment 3 - epochs=10,steps_per_epoch=20 (Freezing All The Layers Except Last 12 Layers Of The Baseline Model)** 




In [ ]:
reshist = model.fit(train_ds, validation_data = val_ds, steps_per_epoch = 20, epochs = 10)

Epoch 1/10
20/20 [==============================] - 502s 26s/step - loss: 1.3059 - accuracy: 0.3969 - val_loss: 1.2902 - val_accuracy: 0.4263
Epoch 2/10
20/20 [==============================] - 504s 26s/step - loss: 1.2891 - accuracy: 0.4000 - val_loss: 1.2898 - val_accuracy: 0.3988
Epoch 3/10
20/20 [==============================] - 525s 27s/step - loss: 1.2761 - accuracy: 0.4078 - val_loss: 1.2840 - val_accuracy: 0.4075
Epoch 4/10
20/20 [==============================] - 528s 27s/step - loss: 1.3067 - accuracy: 0.3625 - val_loss: 1.2812 - val_accuracy: 0.4087
Epoch 5/10
20/20 [==============================] - 519s 27s/step - loss: 1.2878 - accuracy: 0.3953 - val_loss: 1.2799 - val_accuracy: 0.4275
Epoch 6/10
20/20 [==============================] - 524s 27s/step - loss: 1.2786 - accuracy: 0.3891 - val_loss: 1.2775 - val_accuracy: 0.4225
Epoch 7/10
20/20 [==============================] - 519s 27s/step - loss: 1.2503 - accuracy: 0.4437 - val_loss: 1.2732 - val_accuracy: 0.4275
Epoch 